In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import io
import tarfile
import os

dates_index_path = "ionosphere-data/jpld/webdataset/dates_index_2010-05-13T00:00:00_2024-07-31T23:45:00"
tar_index_path = "ionosphere-data/jpld/webdataset/tar_files_index"

with open(dates_index_path, "rb") as f:
    dates_index = pickle.load(f)
with open(tar_index_path, "rb") as f:
    tar_files_index = pickle.load(f)

def datetime_to_key(dt):
    return dt.strftime("%Y/%m/%d/%H%M.tecmap.npy")

def patch_tar_path(path):
    # Use the local data directory, not the absolute original
    return os.path.join("ionosphere-data/jpld/webdataset", os.path.basename(path))

def plot_vtec_for_datetime(requested_datetime_str):
    import pandas as pd
    requested_dt = pd.to_datetime(requested_datetime_str)
    key = datetime_to_key(requested_dt)
    if key not in tar_files_index:
        print(f"Exact datetime not found for {key}. Searching nearest...")
        keys = list(tar_files_index.keys())
        key_times = [pd.to_datetime(k[:16], format="%Y/%m/%d/%H%M") for k in keys]
        idx = np.argmin(np.abs([(t - requested_dt).total_seconds() for t in key_times]))
        key = keys[idx]
        requested_dt = key_times[idx]
        print(f"Using nearest available: {key}")
    tar_path, tarinfo = tar_files_index[key]
    local_tar_path = patch_tar_path(tar_path)
    with tarfile.open(local_tar_path, "r") as tar:
        member = tarinfo
        file_obj = tar.extractfile(member)
        array_bytes = file_obj.read()
    vtec_arr = np.load(io.BytesIO(array_bytes))
    plt.figure(figsize=(8, 4))
    plt.imshow(vtec_arr, cmap='viridis', origin='lower', aspect='auto')
    plt.xlabel("Longitude Index")
    plt.ylabel("Latitude Index")
    plt.title(f"VTEC @ {requested_dt}")
    plt.colorbar(label="VTEC (TECU)")
    plt.tight_layout()
    plt.show()

# Example usage:
plot_vtec_for_datetime("2017-03-22T04:45:00")


Below will animate for a date range April 9, 2024 for 48 hours.. Then put on sun and dark plots, with moon track. 

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import io
import tarfile
import os
import pandas as pd
import matplotlib.animation as animation
import cartopy.crs as ccrs
from skyfield.api import load, wgs84

# --- Index loading ---
dates_index_path = "ionosphere-data/jpld/webdataset/dates_index_2010-05-13T00:00:00_2024-07-31T23:45:00"
tar_index_path = "ionosphere-data/jpld/webdataset/tar_files_index"
with open(dates_index_path, "rb") as f:
    dates_index = pickle.load(f)
with open(tar_index_path, "rb") as f:
    tar_files_index = pickle.load(f)

def datetime_to_key(dt):
    return dt.strftime("%Y/%m/%d/%H%M.tecmap.npy")

def patch_tar_path(path):
    return os.path.join("ionosphere-data/jpld/webdataset", os.path.basename(path))

def nearest_key(requested_dt):
    key = datetime_to_key(requested_dt)
    if key in tar_files_index:
        return key, requested_dt
    keys = list(tar_files_index.keys())
    key_times = [pd.to_datetime(k[:16], format="%Y/%m/%d/%H%M") for k in keys]
    idx = np.argmin(np.abs([(t - requested_dt).total_seconds() for t in key_times]))
    return keys[idx], key_times[idx]

def load_vtec_arr(dt):
    key, used_dt = nearest_key(dt)
    tar_path, tarinfo = tar_files_index[key]
    local_tar_path = patch_tar_path(tar_path)
    with tarfile.open(local_tar_path, "r") as tar:
        file_obj = tar.extractfile(tarinfo)
        array_bytes = file_obj.read()
    vtec_arr = np.load(io.BytesIO(array_bytes))
    return vtec_arr, used_dt

# --- Skyfield setup for lunar point ---
ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']
moon = eph['moon']

def get_subsolar_lon(dt):
    seconds = dt.hour * 3600 + dt.minute * 60 + dt.second
    frac_of_day = seconds / 86400.0
    return (frac_of_day * 360.0) - 180.0

def get_moon_lonlat(dt):
    t = ts.utc(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
    subpoint = wgs84.subpoint(earth.at(t).observe(moon))
    lon = subpoint.longitude.degrees
    lat = subpoint.latitude.degrees
    if lon > 180: lon -= 360
    if lon < -180: lon += 360
    return lon, lat

def wrap180(x):
    return ((x + 180) % 360) - 180

# --- Animation setup ---
start_date_str = "2024-04-09T00:00:00"
start_dt = pd.to_datetime(start_date_str)
n_steps = 48 * 4 + 1  # 48 hours, 15 min steps

dt_list = [start_dt + pd.Timedelta(minutes=15) * i for i in range(n_steps)]

vtec_sample, _ = load_vtec_arr(dt_list[0])
nlat, nlon = vtec_sample.shape
lats = np.linspace(-87.5, 87.5, nlat)
lons = np.linspace(-180, 180, nlon)

# --- Set fixed color range ---
vtec_vmin = 0
vtec_vmax = 125

fig = plt.figure(figsize=(16, 8))
plt.rcParams['animation.embed_limit'] = 50_000_000

def update(frame_idx):
    plt.clf()
    dt = dt_list[frame_idx]
    vtec_arr, used_dt = load_vtec_arr(dt)
    subsolar_lon = get_subsolar_lon(used_dt)
    nightside_lon = wrap180(subsolar_lon + 180)
    moon_lon, moon_lat = get_moon_lonlat(used_dt)

    # Compute lunar position in sun-locked frame
    moon_lon_sunlocked = wrap180(moon_lon - subsolar_lon)

    # --- Sun-facing globe ---
    ax1 = plt.subplot(1, 2, 1, projection=ccrs.Orthographic(central_longitude=-subsolar_lon, central_latitude=0))
    ax1.set_global()
    ax1.coastlines()
    mesh1 = ax1.pcolormesh(
        lons, lats, vtec_arr, cmap='gist_ncar', vmin=vtec_vmin, vmax=vtec_vmax,
        shading='auto', transform=ccrs.PlateCarree()
    )
    ax1.set_title(f"Sun-facing Side\nSubsolar Lon: {subsolar_lon:.1f}°", fontsize=12)
    # Do not plot moon on the dayside globe

    # --- Nightside globe ---
    ax2 = plt.subplot(1, 2, 2, projection=ccrs.Orthographic(central_longitude=-nightside_lon, central_latitude=0))
    ax2.set_global()
    ax2.coastlines()
    mesh2 = ax2.pcolormesh(
        lons, lats, vtec_arr, cmap='gist_ncar', vmin=vtec_vmin, vmax=vtec_vmax,
        shading='auto', transform=ccrs.PlateCarree()
    )
    ax2.plot(moon_lon_sunlocked + nightside_lon, moon_lat, 'o', markersize=16, color='yellow',
             markeredgecolor='black', markeredgewidth=2,
             transform=ccrs.PlateCarree(), label='Sublunar Point')
    ax2.set_title(f"Dark Side\nOpposite Lon: {nightside_lon:.1f}°", fontsize=12)

    # Shared colorbar below both (fixed 0 to 125)
    cbar_ax = fig.add_axes([0.25, 0.10, 0.5, 0.03])
    plt.colorbar(mesh1, cax=cbar_ax, orientation='horizontal', label='VTEC (TECU)')
    fig.suptitle(f'VTEC | {used_dt.strftime("%Y-%m-%d %H:%M:%S UTC")}', fontsize=15)
    return [mesh1, mesh2]

ani = animation.FuncAnimation(fig, update, frames=len(dt_list), interval=150, blit=False)
ani.save('vtec_orthographic_day_night_moon.gif', writer='pillow', fps=6)
print("Saved vtec_orthographic_day_night_moon.gif")

from IPython.display import HTML
HTML(ani.to_jshtml())
